### Please install the required Python modules/SDKs

In [ ]:
! activate ai-azure-c1

import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

### This demo uses the latest pillow package to show the rectangular bounding box around the face, so please upgrade the pillow package using the command below:

In [ ]:
!pip install Pillow==8.4

## Importing Useful Python Libraries or Packages

In [ ]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw


from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person


import matplotlib.pyplot as plt

## Utility Functions

In [ ]:
def show_image_in_cell(face_url):
    response = requests.get(face_url)
    img = Image.open(BytesIO(response.content))
    plt.figure(figsize=(20,10))
    plt.imshow(img)
    plt.show()

In [ ]:
def show_image_object_in_cell(image_object):
    plt.figure(figsize=(20,10))
    plt.imshow(image_object)
    plt.show()

In [ ]:
# TAKEN FROM THE Azure SDK Sample
# Convert width height to a point in a rectangle
def getRectangle(faceDictionary):
    rect = faceDictionary.face_rectangle
    left = rect.left
    top = rect.top
    right = left + rect.width
    bottom = top + rect.height
    
    return ((left, top), (right, bottom))

In [ ]:
def drawFaceRectangles(source_file, detected_face_object) :
    # Download the image from the url
    response = requests.get(source_file)
    img = Image.open(BytesIO(response.content))
    # Draw a red box around every detected faces
    draw = ImageDraw.Draw(img)
    for face in detected_face_object:
        draw.rectangle(getRectangle(face), outline='red', width = 10)
    return img

## Accessing Specific Azure Resources 

In [ ]:
KEY = "ENTER FACE SERVICE RESOURCE KEY"
ENDPOINT = "ENTER FACE SERVICE RESOURCE ENDPOINT"

In [ ]:
# Create an authenticated FaceClient.
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

## Setting up Input Data

### You can use your own images or use the following publically-available images:

In [ ]:
face_x = "https://raw.githubusercontent.com/udacity/cd0461-building-computer-vision-solutions-with-azure-exercises/main/resources/obama-photo.jpg"
face = "https://raw.githubusercontent.com/udacity/cd0461-building-computer-vision-solutions-with-azure-exercises/main/resources/face-portrait.jpg"
family = "https://raw.githubusercontent.com/udacity/cd0461-building-computer-vision-solutions-with-azure-exercises/main/resources/obama_family.jpg"
family_x = "https://raw.githubusercontent.com/udacity/cd0461-building-computer-vision-solutions-with-azure-exercises/main/resources/obama-inauguration.jpg"


In [ ]:
show_image_in_cell(face)

In [ ]:
show_image_in_cell(family_x)

In [ ]:
source_image = face

## Face Detection Functions Using Azure Face Service Python SDK

In [ ]:
# Detect Face form an image
def detect_face_from_any_url(selected_image):
    detected_faces = face_client.face.detect_with_url(url=selected_image, detection_model='detection_03')
    if not detected_faces:
        raise Exception('No face detected from image {}'.format(single_image_name))        
    print('Total face(s) detected  from {}'.format(str(len(detected_faces))))
    return detected_faces

In [ ]:
def list_all_faces_from_detected_face_object(detected_faces_object):
    print('We found total {} face(s) in selected face detected object.'.format(str(len(detected_faces_object))))
    for face in detected_faces_object: 
        print (face.face_id)

## Detecting faces from the input source image

In [ ]:
source_faces_object = detect_face_from_any_url(source_image)

In [ ]:
list_all_faces_from_detected_face_object(source_faces_object)

## Showing bounding box around the detected face

In [ ]:
drawFaceRectangles(source_image, source_faces_object)

In [ ]:
selected_image_2 = family_x

In [ ]:
group_faces_object = detect_face_from_any_url(selected_image_2)

In [ ]:
list_all_faces_from_detected_face_object(group_faces_object)

In [ ]:
drawFaceRectangles(selected_image_2, group_faces_object)

In [ ]:
for face in source_faces_object:
    source_image_face_id = face.face_id

In [ ]:
print(source_image_face_id)

In [ ]:
group_image_face_IDs_list = list(map(lambda x: x.face_id, group_faces_object))

In [ ]:
print('All faces in the group list {}'.format(str(len(group_image_face_IDs_list))))

## Find Similar API

In [ ]:
similar_faces = face_client.face.find_similar(face_id=source_image_face_id, face_ids=group_image_face_IDs_list)

In [ ]:
for similar_face in similar_faces:
    print(similar_face.face_id)

In [ ]:
for face in similar_faces:
		first_image_face_ID = face.face_id
		# The similar face IDs of the single face image and the group image do not need to match, 
		# they are only used for identification purposes in each image.
		# The similar faces are matched using the Cognitive Services algorithm in find_similar().
		face_info = next(x for x in group_faces_object if x.face_id == first_image_face_ID)
		if face_info:
			print('  Face ID: ', first_image_face_ID)
			print('    face_id: ', str(face_info.face_id))
			print('  Face rectangle:')
			print('    Left: ', str(face_info.face_rectangle.left))
			print('    Top: ', str(face_info.face_rectangle.top))
			print('    Width: ', str(face_info.face_rectangle.width))
			print('    Height: ', str(face_info.face_rectangle.height))

In [ ]:
def get_similar_face_object(similar_faces, group_faces_object):
    for face in similar_faces:
        first_image_face_ID = face.face_id
        face_info = next(x for x in group_faces_object if x.face_id == first_image_face_ID)
        if face_info:
            return face_info
        return None

In [ ]:
similar_face_info = get_similar_face_object(similar_faces, group_faces_object)

In [ ]:
response = requests.get(selected_image_2)
img = Image.open(BytesIO(response.content))
# Draw a red box around every detected faces
draw = ImageDraw.Draw(img)
draw.rectangle(getRectangle(similar_face_info), outline='red', width = 10)
plt.figure(figsize=(20,10))
plt.imshow(img)
plt.show()


## Face - Verify API (Example)

In [ ]:
verify_result_same = face_client.face.verify_face_to_face(source_image_face_id, first_image_face_ID)

In [ ]:
print('Faces from {} & {} are of the same person, with confidence: {}'.format(source_image, selected_image_2, verify_result_same.confidence))
if verify_result_same.is_identical:
      print("Faces are Similar")
else:
      print('Faces from {} & {} are of a different person, with confidence: {}'.format(source_image, selected_image_2, verify_result_same.confidence))